<a href="https://colab.research.google.com/github/alyagabsi/inventory-prediction/blob/main/Optimization_Section/Order_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ordering Optimization Problem**
_______________

Please see the formulation of the problem [here](https://drive.google.com/file/d/1XMsGGo5HXC9JMXMm8wkVrzbxwz5PSyob/view?usp=sharing).

## **Setting Dependencies**

In [ ]:
# Importing packages 

!pip install pulp
import pandas as pd 
import numpy as np
from pulp import * 

In [38]:
# User input 
products = ['Apples', 'Oranges', 'Potatoes']
units = {'Apples': 'lbs',
         'Oranges':'lbs',
         'Potatoes': 'lbs'}

C = {} # quote per unit
I = {} # supplier inventory
G = {} # produced in garden
CL = {} # carry over leftovers
L = {} # allowable leftovers

for product in products:
  print('Please input the following information for ' + product + ':')
  print('')
  C[product] = float(input('Supplier quote per ' + units[product]+': '))
  print('')
  I[product] = float(input('Supplier inventory in ' + units[product]+ ': '))
  print('')
  G[product] = float(input('Supply of grown ' + product + ' in ' + units[product]+ ': '))
  print('')
  CL[product] = float(input('Carry over leftovers in '+ units[product] + ': '))
  print('')
  L[product] = float(input('Allowable leftovers in '+ units[product] + ': '))
  print('')

# Demand (given by forecasting models) - TBDone

D = {'Apples':10,
     'Oranges': 10,
     'Potatoes': 10}



Please input the following information for Apples:

Supplier quote per lbs: 10

Supplier inventory in lbs: 10

Supply of grown Applesin lbs: 10

Carry over leftovers in lbs: 10

Allowable leftovers in lbs: 10

Please input the following information for Oranges:

Supplier quote per lbs: 10

Supplier inventory in lbs: 10

Supply of grown Orangesin lbs: 10

Carry over leftovers in lbs: 10

Allowable leftovers in lbs: 10

Please input the following information for Potatoes:

Supplier quote per lbs: 10

Supplier inventory in lbs: 10

Supply of grown Potatoesin lbs: 10

Carry over leftovers in lbs: 10

Allowable leftovers in lbs: 10



## **Formulating the Problem**

In [55]:
# Initializing model

model = LpProblem('Order_Optimization', LpMinimize)

Setting Decision Variables 

In [56]:
# Amount to order from supplier 

X = LpVariable.dicts('Order', products, lowBound = 0, cat = 'Continuous')

#  Sales 

S = LpVariable.dicts('Sales', products, lowBound =0, cat='Continuous')

In [53]:
X

{'Apples': Order_Apples, 'Oranges': Order_Oranges, 'Potatoes': Order_Potatoes}

Setting Objective Function

In [57]:
model += sum([X[product]*C[product] for product in products])

Setting Constraints

In [58]:
# Demand Constraints

for product in products:
  model += X[product] + G[product] + CL[product] >= D[product] # demand constraints
  model += -X[product] >= -I[product] # supplier constraints
  model += -S[product] >= -D[product] # sales constraints
  model += -(X[product] + G[product] + CL[product] - S[product]) >= -L[product] # leftover constraints

In [59]:
print(model)

Order_Optimization:
MINIMIZE
10.0*Order_Apples + 10.0*Order_Oranges + 10.0*Order_Potatoes + 0.0
SUBJECT TO
_C1: Order_Apples >= -10

_C2: - Order_Apples >= -10

_C3: - Sales_Apples >= -10

_C4: - Order_Apples + Sales_Apples >= 10

_C5: Order_Oranges >= -10

_C6: - Order_Oranges >= -10

_C7: - Sales_Oranges >= -10

_C8: - Order_Oranges + Sales_Oranges >= 10

_C9: Order_Potatoes >= -10

_C10: - Order_Potatoes >= -10

_C11: - Sales_Potatoes >= -10

_C12: - Order_Potatoes + Sales_Potatoes >= 10

VARIABLES
Order_Apples Continuous
Order_Oranges Continuous
Order_Potatoes Continuous
Sales_Apples Continuous
Sales_Oranges Continuous
Sales_Potatoes Continuous



## **Solving LP**

In [ ]:
result = model.solve()

Printing Results



In [ ]:
print('Optimial Values:')
print('')
for var in model.variables():
  print(var.name, '==>', var.varValue)

print('')
print('Expected Total cost: $',pulp.value(model.objective))